# SECTION 1: DATA CURATION
___

Our first task is to find data for our project. We will do this through the use of Google's YouTube Data API v3.

Unfortunately, we can't just use the channel names that the user can see on the site when calling the API, since not all channels have a channelName, depending on when they were created. Not to mention that there are copius amounts of copycat accounts out there with similar names to their real counterparts. The way around this is to obtain a set of channelId's, which all channels have, and serve as unique identifiers; channelId's are also required for interfacing with most of the APIs features and methods. To find a given channel's ID, we have to manually visit a channel's home page, and extract it from the home page's source code.

Since this is a manual process, we will just append these IDs to lists in the file `channel_ids.py` and import that into the following code.


## Outline

| Part | Task                                                       |
|------|------------------------------------------------------------|
| I    | Gather sample of channels                                  |
| II   | Gather sample of videos from each channel                  |
| III  | Gather sample of comments from each video                  |
| IV   | remove nulls + gather more ids if necessary + repeat I-III |


In [8]:
import os
import googleapiclient.discovery
from googleapiclient.discovery import HttpError
import pandas as pd
from pathlib import Path
from os import listdir, getcwd
from importlib import reload
pd.options.mode.chained_assignment = None

In [27]:
# authentication step
import api_keys
developer_key = api_keys.developer_key

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=developer_key)

### **PART I**: SAMPLING CHANNELS

In [29]:
# channel ids obtained from their channel's page source code
import channel_ids
reload(channel_ids)

<module 'channel_ids' from 'C:\\Users\\Matt\\DataspellProjects\\Youtube Data Analytics\\channel_ids.py'>

In [34]:
def get_channel_details(channel_ids) -> pd.DataFrame | None:
    """retrieves various details about the provided channel(s), like channel name, id, sub count, video count, view count, etc."""
    # limit is 50 channels_ids at a time
    assert len(channel_ids) <= 50
    all_data = []
    request = youtube.channels().list(part="snippet,contentDetails,statistics", id=",".join(channel_ids))
    response = request.execute()

    for item in response['items']:
        data = {"channelId": item['id'],
                "channelName": item['snippet']['title'],
                "viewCount": item['statistics']['viewCount'],
                "subscriberCount": item['statistics']['subscriberCount'],
                "videoCount": item['statistics']['videoCount'],
                "uploads": item['contentDetails']['relatedPlaylists']['uploads']
                }
        all_data.append(data)
    return pd.DataFrame(all_data)

In [71]:
from channel_ids import new_channel_ids

save_to_new_file: bool = True
new_file_name = 'videos.csv'
new_file_name_2 = 'channels.csv'

new_ids_1, new_ids_2 = new_channel_ids[:50], new_channel_ids[50:]
len(new_ids_1), len(new_ids_2)

(50, 31)

In [37]:
# call with the sublists then merge into one dataframe
df = get_channel_details(new_ids_1)
df2 = get_channel_details(new_ids_2)

In [39]:
df = pd.concat([df, df2], ignore_index=True)
df.sample(5)

,channelId,channelName,viewCount,subscriberCount,videoCount,uploads
0,UCbpMy0Fg74eXXkvxJrtEn3w,Bon Appétit,1806695724,6380000,1543,UUbpMy0Fg74eXXkvxJrtEn3w
1,UCupvZG-5ko_eiXAupbDfxWw,CNN,14843457695,15700000,161812,UUupvZG-5ko_eiXAupbDfxWw
2,UC7_gcs09iThXybpVgjHZ_7g,PBS Space Time,428817752,2960000,367,UU7_gcs09iThXybpVgjHZ_7g
3,UCIEv3lZ_tNXHzL3ox-_uUGQ,Gordon Ramsay,3941028890,20200000,1585,UUIEv3lZ_tNXHzL3ox-_uUGQ
4,UCeY0bbntWzzVIaj2z3QigXg,NBC News,6348059328,9070000,57204,UUeY0bbntWzzVIaj2z3QigXg
...,...,...,...,...,...,...
76,UClFSU9_bUb4Rc6OYfTt5SPw,Philip DeFranco,1928011589,6480000,1748,UUlFSU9_bUb4Rc6OYfTt5SPw
77,UCwWhs_6x42TyRM4Wstoq8HA,The Daily Show,5405189122,10500000,5405,UUwWhs_6x42TyRM4Wstoq8HA
78,UCoLUji8TYrgDy74_iiazvYA,Jarvis Johnson,239388514,2040000,146,UUoLUji8TYrgDy74_iiazvYA
79,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,16674697235,59800000,400,UURijo3ddMTht_IHyNSNXpNQ


In [42]:
# check all the correct channels were obtained by their sub count (we can assume they're correct if they have at least 1 mil subs)
assert all(int(i) > 1000000 for i in df['subscriberCount'].to_list())

By collecting the 'uploads' playlistIds for our set of creators, we can easily access their most recent posted videos, and gather their respective IDs.

### **PART II**: SAMPLING VIDEOS

In [43]:
uploads_ids = df['uploads'].tolist()

In [44]:
def last_n_video_ids(playlist_id, n: int = 50):
    """retrieve the video ids for the creator's last n uploads, provided the channel's 'uploads' playlist ID"""
    ids = []

    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = n
    )

    response = request.execute()

    for item in response['items']:
        ids.append(item['contentDetails']['videoId'])

    return ids

In [45]:
data = []
for playlist_id in uploads_ids:
    video_ids = last_n_video_ids(playlist_id, 50)
    data.append(video_ids)

Now, since commas aren't a valid video ID character, we will transform each sublist of video IDs into a comma-separated string. From that, we will have a simple list of strings we can set as a new column in our dataframe.

In [46]:
videos = [",".join(sub_list) for sub_list in data]
df['videoIDs'] = videos

In [47]:
channel_ids = df['channelId'].tolist()

In [49]:
def get_video_details(video_ids):
    """retrieves information on each video provided, as a list of dicts"""
    all_data = []
    if isinstance(video_ids, list):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(video_ids)
        )
    elif isinstance(video_ids, str):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_ids
        )
    response = request.execute()

    for item in response['items']:
        video_info = {
            'videoId': item['id'],
            'channelName': item['snippet']['channelTitle'],
            'description': item['snippet']['description'],
            'videoTitle': item['snippet']['title'],
            'postDate': item['snippet']['publishedAt'],
            'duration': item['contentDetails']['duration'],
            'views': item['statistics']['viewCount'],
            'commentCount': None,
            'thumbnail': item['snippet']['thumbnails']['default']['url']}
        all_data.append(video_info)
    return all_data

In [50]:
def get_video_ids(channel_id):
    """returns a list of video IDs for the associated channel ID, in the form of a list of strings"""
    video_ids = df[df['channelId'] == channel_id]['videoIDs'].values[0]
    return video_ids.split(',')

### **PART III**: SAMPLING COMMENTS

In [59]:
video_details_list = []

for channel_id in channel_ids:
    video_ids = get_video_ids(channel_id)
    video_details = get_video_details(video_ids)
    video_details_list.extend(video_details)

video_df = pd.DataFrame(video_details_list)

The result of the code above will allow us to cross-reference information between our channel df and the newly created ```video_df```, since we can connect video Ids to channel Ids. As we will see soon, this will become more important after we store comments from these videos in a new dataframe, called ```comments_df```.

In [27]:
def get_n_comments(video_id, n=50):
    comments = []
    try:
        results = youtube.commentThreads().list(
            part = 'snippet',
            videoId = video_id,
            textFormat = 'plainText',
            maxResults=n
        ).execute()
    except HttpError:
        # comments are disabled on video
        return [None]
    else:
        for item in results['items']:
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            comments.append(comment)
        return comments

In [28]:
# create nested dict of comment data:
# dict of channel_ids -> dict of video_ids -> list of comments

# all_comment_data = {}
#
# for channel_id in channel_ids:
#     comment_data = {}
#     channel_video_ids = get_video_ids(channel_id)
#     for video_id in channel_video_ids:
#         video_comments = get_n_comments(video_id)
#         comment_data[video_id] = video_comments
#     all_comment_data[channel_id] = comment_data

In [29]:
# len(all_comment_data.keys()) == len(channel_ids)

True

In [30]:
# unpack all video_id: comment list pairs into a single df across all creators

# channel_ids = []
# video_ids = []
# comments = []
#
# for channel_id, video_id_dict in all_comment_data.items():
#     for video_id, comments_list in video_id_dict.items():
#         for comment in comments_list:
#             channel_ids.append(channel_id)
#             video_ids.append(video_id)
#             comments.append(comment)
#
# comments_df = pd.DataFrame({'channelId': channel_ids, 'videoId': video_ids, 'comment': comments})

In [60]:
video_df

,videoId,channelName,description,videoTitle,postDate,duration,views,commentCount,thumbnail
0,V1hN1ekwTP8,Bon Appétit,Andrew Rea (AKA Binging With Babish) pits 32 c...,Babish Picks the Best Halloween Candy of All-T...,2023-10-24T16:00:02Z,PT17M33S,1214,None,https://i.ytimg.com/vi/V1hN1ekwTP8/default.jpg
1,d8q6bC-pAF8,Bon Appétit,"Japanese chef Yuji Haraguchi, owner of OKONOMI...",Flaming Fish Chashu,2023-10-20T18:45:02Z,PT58S,31681,None,https://i.ytimg.com/vi/d8q6bC-pAF8/default.jpg
2,hktIRdd90g4,Bon Appétit,Kendra Vaculin spent days in the Bon Appétit T...,Developing These Perfect Lemon Bars Nearly Bro...,2023-10-19T16:45:02Z,PT24M29S,140798,None,https://i.ytimg.com/vi/hktIRdd90g4/default.jpg
3,Jk90CG3WBy8,Bon Appétit,Chef Evan Funke brings Bon Appétit along to th...,Artichoke Hips Don't Lie,2023-10-18T20:15:02Z,PT27S,38809,None,https://i.ytimg.com/vi/Jk90CG3WBy8/default.jpg
4,9Tc33xCppQo,Bon Appétit,"“We make about 13 to 14,000 cookies every week...","Making 28,000 Pastries a Week in a Small Brook...",2023-10-17T16:00:07Z,PT20M1S,171404,None,https://i.ytimg.com/vi/9Tc33xCppQo/default.jpg
...,...,...,...,...,...,...,...,...,...
4045,CIc8fHhO5O0,Ed Sheeran,Autumn Variations out now: https://es.lnk.to/a...,Autumn Is Coming #4,2023-09-18T22:00:06Z,PT48S,130324,None,https://i.ytimg.com/vi/CIc8fHhO5O0/default.jpg
4046,m9JM-a5AH54,Ed Sheeran,Subscribe to Ed's channel: http://bit.ly/Subsc...,"Santa Clara pop up before the stadium show, gi...",2023-09-18T14:08:11Z,PT41S,124038,None,https://i.ytimg.com/vi/m9JM-a5AH54/default.jpg
4047,1btffD7DYVQ,Ed Sheeran,Subscribe to Ed's channel: http://bit.ly/Subsc...,Debuting American Town unplugged in the audien...,2023-09-16T09:18:26Z,PT1M,93226,None,https://i.ytimg.com/vi/1btffD7DYVQ/default.jpg
4048,e_irHEmAkgw,Ed Sheeran,Autumn Variations is the first album I’m putti...,I want you guys to make the videos for Autumn ...,2023-09-13T18:39:51Z,PT25S,107355,None,https://i.ytimg.com/vi/e_irHEmAkgw/default.jpg


In [63]:
# save all 3 dfs as csv files in the 'data' folder
root = Path(getcwd())
data_folder = root.joinpath('data')

if save_to_new_file:
    df.to_csv(data_folder.joinpath(new_file_name_2), index=False)
    video_df.to_csv(data_folder.joinpath(new_file_name), index=False)
    print(f"Saved data to files '{new_file_name_2}' '{new_file_name}'")
else:
    file_name = 'videos.csv'
    file_name_2 = 'channels.csv'
    video_df.to_csv(data_folder.joinpath(file_name), mode='a', index=False, header=False)
    df.to_csv(data_folder.joinpath(file_name_2), mode='a', index=False, header=False)
    print(f"Appended to files '{file_name_2}' and '{file_name}'")

Saved data to file 'videos.csv'


In [68]:
# preview results
data = pd.read_csv(data_folder.joinpath('videos.csv'))
new_df = pd.DataFrame(data)
new_df.sample(20)

,videoId,channelName,description,videoTitle,postDate,duration,views,commentCount,thumbnail
389,KrE6S47uqYM,Rhett & Link,NaN,Rhett & Link Live Stream,2017-04-24T23:11:56Z,P0D,0,NaN,https://i.ytimg.com/vi/KrE6S47uqYM/default_liv...
1796,aU53t7EH_CQ,VICE,After years as one of the most successful cred...,How a Jewelry Heist Went Wrong #shorts,2023-08-14T15:00:01Z,PT51S,16896,NaN,https://i.ytimg.com/vi/aU53t7EH_CQ/default.jpg
2355,A1TaT6ny1G0,Good Mythical Morning,"Today, we're comparing groceries from around t...",Which Country Has The Most Expensive Groceries?,2023-10-19T10:00:10Z,PT20M51S,798662,NaN,https://i.ytimg.com/vi/A1TaT6ny1G0/default.jpg
3322,thOifuHs6eY,CGP Grey,‣ Behind the Scenes: https://www.youtube.com/w...,Hexagons are the Bestagons,2020-11-03T16:30:11Z,PT9M27S,12251281,NaN,https://i.ytimg.com/vi/thOifuHs6eY/default.jpg
3737,B6_iQvaIjXw,ArianaGrandeVevo,The official “34+35” music video by Ariana Gra...,Ariana Grande - 34+35 (official video),2020-11-17T17:00:03Z,PT3M41S,300402271,NaN,https://i.ytimg.com/vi/B6_iQvaIjXw/default.jpg
2090,O7hhNl4sR-4,Be Smart,Thank you to Brilliant for supporting PBS. To ...,The Evolutionary Purpose of Fun,2023-01-31T16:20:11Z,PT9M51S,297117,NaN,https://i.ytimg.com/vi/O7hhNl4sR-4/default.jpg
2065,fJ9qKO8VHFw,Be Smart,Another YouTube #shorts from your favorite sci...,The world’s oldest dog is HOW OLD?!,2023-08-07T22:44:44Z,PT49S,562356,NaN,https://i.ytimg.com/vi/fJ9qKO8VHFw/default.jpg
1386,IdSibCpD76I,emma chamberlain,Thanks to Google for sponsoring a portion of t...,HOW TO ENJOY BOREDOM,2021-05-30T20:30:18Z,PT22M51S,6213026,NaN,https://i.ytimg.com/vi/IdSibCpD76I/default.jpg
3399,Cd3v_pRgOp8,Taylor Swift,Listen to “Anti-Hero (Jayda G remix)” by Taylo...,Taylor Swift - Anti-Hero (Jayda G Remix),2022-11-11T03:00:06Z,PT3M36S,698969,NaN,https://i.ytimg.com/vi/Cd3v_pRgOp8/default.jpg
3599,3J7bosj7sXQ,Drawfee Show,"Nathan, Jacob, Karina, and Julia only have 30 ...",We Only Had 30 Seconds to Draw These Popular C...,2023-04-20T17:01:17Z,PT25M15S,270175,NaN,https://i.ytimg.com/vi/3J7bosj7sXQ/default.jpg


,channelId,channelName,viewCount,subscriberCount,videoCount,uploads,videoIDs
0,UCbpMy0Fg74eXXkvxJrtEn3w,Bon Appétit,1806695724,6380000,1543,UUbpMy0Fg74eXXkvxJrtEn3w,"V1hN1ekwTP8,d8q6bC-pAF8,hktIRdd90g4,Jk90CG3WBy..."
1,UCupvZG-5ko_eiXAupbDfxWw,CNN,14843457695,15700000,161812,UUupvZG-5ko_eiXAupbDfxWw,"dCgI8OoeZF8,LmVBYsZ4Cp8,gX_PhI5t3kE,5QqFweh3CC..."
2,UC7_gcs09iThXybpVgjHZ_7g,PBS Space Time,428817752,2960000,367,UU7_gcs09iThXybpVgjHZ_7g,"5e83QRGoRCo,ZLrMwDD8u_Q,BUHW1zlstVk,npynpWkUSY..."
3,UCIEv3lZ_tNXHzL3ox-_uUGQ,Gordon Ramsay,3941028890,20200000,1585,UUIEv3lZ_tNXHzL3ox-_uUGQ,"UNW_RWcqZeQ,-4o1IUdTojc,dvepSlTZh1c,XyqzcfTWyg..."
4,UCeY0bbntWzzVIaj2z3QigXg,NBC News,6348059328,9070000,57204,UUeY0bbntWzzVIaj2z3QigXg,"GE4FXNDAxG8,TsjVbJH6KXI,NkIeeHYzNpo,ubderM7G4h..."
...,...,...,...,...,...,...,...
76,UClFSU9_bUb4Rc6OYfTt5SPw,Philip DeFranco,1928011589,6480000,1748,UUlFSU9_bUb4Rc6OYfTt5SPw,"sO8HD0YxNZk,j110aDhWHyY,KFbxSSz7wPY,t5koy7769z..."
77,UCwWhs_6x42TyRM4Wstoq8HA,The Daily Show,5405189122,10500000,5405,UUwWhs_6x42TyRM4Wstoq8HA,"vbCSiHc-szY,LDPFOVvSFDU,nMKL-LyTVn8,6FpfzjTOEs..."
78,UCoLUji8TYrgDy74_iiazvYA,Jarvis Johnson,239388514,2040000,146,UUoLUji8TYrgDy74_iiazvYA,"KW64FiB0ITg,Gw_r47vQ2Sw,SUGfvEAlvk0,2wM3pf79QS..."
79,UCRijo3ddMTht_IHyNSNXpNQ,Dude Perfect,16674697235,59800000,400,UURijo3ddMTht_IHyNSNXpNQ,"ZVPpl_sV97U,cA-DTIpVdH0,UGz1AbmJIuU,9o__c8wNd3..."
